# Algoritmo final de clasificación

En este notebook pondremos a prueba el algoritmo, simulando la entrada de nuevos registros para su clasificación, haciendo la salvedad de que los nuevos registros provienen de tiendas no incluidas en el dataset de entrenamiento ni validación de los modelos. Lo anterior con el fin de asegurarnos de que los modelos no padezcan de sobreajuste.

Así mismo, formularemos el algoritmo completo de clasificación que servirá de entregable final para la formulación de un prototipo dentro del área de analítica de Bancolombia. 

Iniciaremos con la configuración del ambiente de ejecución.

In [1]:
# Loads Constants, imports and utilitary functions
%run 0_Utils.ipynb

In [1]:
%%capture
!pip install tensorflow
import boto3
import io
from PIL import Image               
from IPython.display import display 
import json
import pandas as pd
import plotly.express as px
import os
from botocore import UNSIGNED
from botocore.client import Config
import joblib
from tensorflow import keras
from tensorflow.keras import layers
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')

Ahora, definiremos unas funciones que irán embebidas en el algoritmo final, y que nos permiten llevar a cabo la extracción del texto de las imágenes, junto con el total, y el preprocesamiento de dicho texto para ingresarlo a los modelos.

In [2]:
def extraer (ruta_imagen):
    client = boto3.client("textract", region_name=REGION_NAME)

    with open (ruta_imagen, 'rb') as ima:
        response = client.detect_document_text(Document={'Bytes':ima.read()})
        image = Image.open(ima)
        pixels = image.load()
    
    blocks = response['Blocks']
    line_list = []
    for block in blocks:
            if block["BlockType"] == "LINE":
                line_list.append(block["Text"])
                
    texto = str (line_list)

    with open (ruta_imagen, 'rb') as ima:
        response1 = client.analyze_expense(Document={'Bytes':ima.read()}) 
    try:
        for summary_field in response1['ExpenseDocuments'][0]['SummaryFields']:

            if "Type" in summary_field and summary_field['Type']['Text'] == 'TOTAL':
                total = summary_field['ValueDetection']['Text']
    except:
        total = ""
    
    try: total
    except NameError: total = ""   
    
    
    return texto, total, image

In [3]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('spanish')]
    return " ".join(filtered_words)

Una vez definidas las funciones embebidas, procederemos a definir el algoritmo de clasificación, el cual recibe como entrada una ruta de la imagen en formato JPG o PNG, y la especificación del modelo a utilizar para la clasificación (perceptron o svm), y retorna la imagen en cuestión junto con la categoría a la cual pertenece la imágen.

In [4]:
def algoritmo (ruta_imagen, modelo): # modelo = perceptron o svm
    
    texto, total, imagen = extraer (ruta_imagen)
    texto = preprocess(texto)
    
    vectorizer = joblib.load("TFIDF-vectorizer.joblib")
    X = vectorizer.transform([texto])
    X = X.todense()
    
    if modelo == 'svm':
        modelo_svm = joblib.load("SVM.joblib")
        pred = modelo_svm.predict(X)[0]

    elif modelo == 'perceptron':
        modelo_perceptron = keras.models.load_model("Perceptron.h5")
        pred = modelo_perceptron.predict(X)
        
        maxindex = np.argmax(pred)
        lst = ['Alimentación', 'Grande superficie', 'Moda', 'No factura', 'Salud y Bienestar']
        
        pred = lst[maxindex]
    
    else:
        pred = 'Por favor indicar un modelo en la función entre perceptron o svm'
        
    return pred, total, imagen

Ahora bien, para llevar a cabo la comprobación, aplicaremos el algoritmo a cinco imágenes nuevas subidas al ambiente de ejecución de SageMaker.

In [5]:
pruebas = ['Prueba_no_Fact.jpg', 'Prueba_no_texto.png', 'Prueba_alimentacion.JPG', 'Prueba_moda.JPG', 'Prueba_salud.JPG']

In [6]:
%%capture --no-stdout

for i in pruebas:
    print (i, algoritmo(i,'svm')[0:2])

Prueba_no_Fact.jpg ('No factura', '$ 26.367.392')
Prueba_no_texto.png ('No factura', '')
Prueba_alimentacion.JPG ('Alimentación', '$25.300')
Prueba_moda.JPG ('Moda', '239.800')
Prueba_salud.JPG ('Salud y Bienestar', '$ 43.300')


In [7]:
%%capture --no-stdout

for i in pruebas:
    print (i, algoritmo(i,'perceptron')[0:2])

Prueba_no_Fact.jpg ('No factura', '$ 26.367.392')
Prueba_no_texto.png ('No factura', '')
Prueba_alimentacion.JPG ('Alimentación', '$25.300')
Prueba_moda.JPG ('Moda', '239.800')
Prueba_salud.JPG ('Salud y Bienestar', '$ 43.300')


Como se puede observar, el algoritmo, tanto con el modelo Linear Support Vector Classifier como con el Perceptrón Multicapa, aplicado a las cinco imágenes logra clasificar adecuadamente la categoría respectiva. Así mismo, se logra identificar el total de la factura adecuadamente.

In [18]:
%%time
algoritmo(pruebas[2],'perceptron')[0:2]

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning:

Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning:

Trying to unpickle estimator TfidfTransformer from version 1.0.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning:

Trying to unpickle estimator TfidfVectorizer from version 1.0.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.



CPU times: user 700 ms, sys: 97.5 ms, total: 798 ms
Wall time: 6.29 s


('Alimentación', '$25.300')

In [19]:
%%time
algoritmo(pruebas[2],'svm')[0:2]

CPU times: user 534 ms, sys: 22.3 ms, total: 556 ms
Wall time: 6.18 s


('Alimentación', '$25.300')

### Desgloce del tiempo de ejecución

Extracción del texto:

In [11]:
%%time
texto, total, imagen = extraer (pruebas[2])

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning:

Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/



CPU times: user 466 ms, sys: 29.2 ms, total: 495 ms
Wall time: 6.25 s


Preprocesamiento del texto extraído:

In [12]:
%%time
texto = preprocess(texto)

CPU times: user 17.9 ms, sys: 25 µs, total: 17.9 ms
Wall time: 22.5 ms


Vectorización del texto preprocesado:

In [13]:
%%time
vectorizer = joblib.load("TFIDF-vectorizer.joblib")
X = vectorizer.transform([texto])
X = X.todense()

CPU times: user 69.9 ms, sys: 0 ns, total: 69.9 ms
Wall time: 81.9 ms


/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning:

Trying to unpickle estimator TfidfTransformer from version 1.0.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning:

Trying to unpickle estimator TfidfVectorizer from version 1.0.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.



Predicción SVM

In [16]:
%%time
modelo_svm = joblib.load("SVM.joblib")
print(modelo_svm.predict(X)[0], total)

Alimentación $25.300
CPU times: user 1.14 ms, sys: 2.98 ms, total: 4.12 ms
Wall time: 5.25 ms


Predicción Perceptrón Multicapa

In [17]:
%%time
modelo_perceptron = keras.models.load_model("Perceptron.h5")
pred = modelo_perceptron.predict(X)

maxindex = np.argmax(pred)
lst = ['Alimentación', 'Grande superficie', 'Moda', 'No factura', 'Salud y Bienestar']
print(lst[maxindex], total)

Alimentación $25.300
CPU times: user 182 ms, sys: 44.8 ms, total: 226 ms
Wall time: 234 ms


Extracción del texto: 6.25 s
Preprocesamiento del texto extraído: 22.5 ms
Vectorización del texto preprocesado: 81.9 ms
Predicción SVM: 5.25 ms
Predicción Perceptrón: 234 ms